In [1]:
import sys
sys.path.append("../../Python")
import ROOT as R
from array import array
import time
import numpy as np
import pandas as pd
import seaborn as sns
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
#from root_helpers import print_mc_particle_tree
#from root_helpers import print_daughters
R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../../Python/Utility_Functions_C.so') - os.path.getmtime('../../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../../Python/Utility_Functions_C.so")

R.gSystem.Load("../lib/libMC2021")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.5 \n'

In [2]:
ch = R.TChain("MiniDST")
#ch.Add("../data/electrons_n1_*.root")
ch.Add("../data/electrons_iss963_n1_*.root")
# ch.Add(f"..//data/HPS/data/physrun2021/sim_2021/hpsForward_e-_{e}GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("..//data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")
df = R.RDataFrame(ch)

Number of events loaded:   5.166M


In [3]:
R.gInterpreter.ProcessLine('''auto EAC = Ecal_Analysis_Class();''')   # This is key. It puts the EAC in C++ space.
print(f"{R.EAC.Version()}")

V1.0.8


In [4]:
dfx = R.EAC.extend_dataframe(R.RDF.AsRNode(df))
dfx = dfx.Define("mc_part_primary_energy","return mc_part_energy[mc_part_primary_index[0]]")  # This is sloppy. It assumes that there is always one and only one primary particle. Okay for this data.

In [5]:
print("Available data names in Tuple:")
ColumnNames=dfx.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if str(x).startswith('')]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy           ecal_cluster_hits             ecal_cluster_nhits            ecal_cluster_seed_energy      
ecal_cluster_seed_index       ecal_cluster_seed_ix          ecal_cluster_seed_iy          ecal_cluster_time             
ecal_cluster_uncor_energy     ecal_cluster_uncor_hits       ecal_cluster_uncor_nhits      ecal_cluster_uncor_seed_energy
ecal_cluster_uncor_seed_index ecal_cluster_uncor_seed_ix    ecal_cluster_uncor_seed_iy    ecal_cluster_uncor_time       
ecal_cluster_uncor_x          ecal_cluster_uncor_y          ecal_cluster_uncor_z          ecal_cluster_x                
ecal_cluster_y                ecal_cluster_z                ecal_hit_energy               ecal_hit_index_x              
ecal_hit_index_y              ecal_hit_time                 ecal_hit_x                    ecal_hit_y                    
ecal_hit_z                    event_number                  ext_trigger                   hodo_cluster_energy           


In [6]:
class Store:
    def __init__(self, name="", short_name="", data_frame=None):
        self.n = name
        self.sn = short_name
        self.df = data_frame



In [7]:
# Filter the dataframe
dfxc = dfx.Define("fiducial_cut","auto f=EAC.fiducial_cut(ecal_cluster_seed_ix,ecal_cluster_seed_iy); return f;")\
    .Filter("for(size_t i=0; i< fiducial_cut.size(); ++i) if( fiducial_cut[i] == 0){ return false;} return true;")
dfx1 = dfx.Filter("ecal_cluster_energy.size() == 1").Filter("mc_score_cluster_indexes.size()==1")
dfx1c =dfx1.Define("fiducial_cut","return EAC.fiducial_cut(ecal_cluster_seed_ix,ecal_cluster_seed_iy);")\
    .Filter("for(size_t i=0; i< fiducial_cut.size(); ++i) if( fiducial_cut[i] == false){ return false;} return true;")
df_store = [Store("No cuts","", dfx),
     Store("Ficucial cut","_fc",dfxc),
     Store("One Cluster", "_1c",dfx1 ),
     Store("One Cluster + Fiducial", "_fc_1c",dfx1c)]

In [8]:
Ones = None
e_levels = [(0.,0.5),(1.0, 1.5), (2.0,2.5), (3.0, 3.5), (4.0, 4.5)]
hh_tmp_xy = []
for i in range(len(e_levels)):
    hh_tmp_xy.append(
        dfx.Filter(f"""mc_score_primary_hits_e.size()>0 && mc_score_primary_hits_e[0] > {e_levels[i][0]} && mc_score_primary_hits_e[0] < {e_levels[i][1]}""")\
        .Histo2D(("hh_tmp_xy", f"Ecal Hits, {e_levels[i][0]} < E < {e_levels[i][1]}; ix; iy", 50, -25.5, 24.5, 13, -6.5, 6.5), "ecal_cluster_seed_ix","ecal_cluster_seed_iy"))


In [9]:
max_i = 20
max_e = 4.8

for s in df_store:
    s.hh = s.df.Histo1D("mc_part_primary_index")
    s.hhn = s.df.Define("n","mc_score_primary_hits_e.size()").Histo1D(("hhn"+s.sn,"Num primary part.;N", max_i, -0.5,max_i - 0.5),"n")
    s.hhn2 = s.df.Define("n","mc_score_secondary_hits_e.size()").Histo1D(("hhn2"+s.sn,"Num secondary part.;N", max_i, -0.5,max_i - 0.5),"n")
    s.hhn_ecal = s.df.Define("n","ecal_cluster_energy.size()").Histo1D(("hhn_ecal"+s.sn,"Number of Ecal Clusters", max_i, -0.5, max_i - 0.5),"n")
    s.hhn_hits = s.df.Histo1D(("hhn_hits"+s.sn,"Number of hits per primary;N", max_i, -0.5,max_i - 0.5),"n_mc_score_primary_hits")
    s.hhn2_hits = s.df.Histo1D(("hhn2_hits"+s.sn,"Number of hits per secondary;N", max_i, -0.5,max_i - 0.5),"n_mc_score_secondary_hits")
    s.hhe = s.df.Histo1D(("hhe"+s.sn,"Energy of Primary Hit on score plane.;E[GeV]",1000,-0.1,max_e),"mc_score_primary_hits_e")
    s.hhe2 = s.df.Histo1D(("hhe2"+s.sn,"Energy of Secondary Hit on score plane.;E[GeV]",1000,-0.1,max_e),"mc_score_secondary_hits_e")
    s.hh_ecalxy = s.df.Histo2D(("hh_ecalxy"+s.sn,"Ecal Cluster (x,y);x;y",100,-400,400,50,-100,100),"ecal_cluster_x","ecal_cluster_y")
    s.hh_ecal_seed_xy = s.df.Histo2D(("hh_ecal_seed_xy"+s.sn,"Ecal Hits; ix;iy",50,-25.5,24.5,13,-6.5,6.5),"ecal_cluster_seed_ix","ecal_cluster_seed_iy")
    s.Ones = None
    s.hhn_clus = s.df.Define("n","return mc_score_cluster_indexes.size()").Histo1D(("hhn_clus"+s.sn,"N score clusters;N", max_i,  -0.5, max_i - 0.5),"n")
    s.hhn_clus_hits = s.df.Define("nn","RVec<int> out;for(size_t i=0; i< mc_score_cluster_indexes.size(); ++i){ out.push_back(mc_score_cluster_indexes[i].size());} return out;").Histo1D(("hhn_clus"+s.sn,"N score clusters;N", max_i,  -0.5, max_i - 0.5),"nn")
    s.hh_diff_x = s.df.Define("d","""RVec<double> out;
    for(size_t i=0; i< min(ecal_cluster_x.size(),mc_score_cluster_x.size()); ++i){
        out.push_back(ecal_cluster_x[i]-mc_score_cluster_x[i]);}
    return out;""").Histo1D(("hh_diff_x"+s.sn,"Ecal_x - Score_x;#Delta x [mm]",1000,-10.,10.),"d")
    s.hh_diff_y = s.df.Define("d","""RVec<double> out;
    for(size_t i=0; i< min(ecal_cluster_y.size(),mc_score_cluster_y.size()); ++i){
        out.push_back(ecal_cluster_y[i]-mc_score_cluster_y[i]);}
    return out;""").Histo1D(("hh_diff_y"+s.sn,"Ecal_y - Score_y;#Delta y [mm]",1000,-10.,10.),"d")
    s.hh_diff_e = s.df.Define("d","""RVec<double> out;
    for(size_t i=0; i< min(ecal_cluster_energy.size(),mc_score_cluster_e.size()); ++i){
        out.push_back(ecal_cluster_energy[i]-mc_score_cluster_e[i]);}
    return out;""").Histo1D(("hh_diff_e"+s.sn,"#Delta Ecal_e Score_e;#Delta E [GeV]",1000,-2.,1.),"d")
    s.hh_diff_e3 = s.df.Define("d","""RVec<double> out;
    for(size_t i=0; i< ecal_cluster_energy.size(); ++i){
        out.push_back(ecal_cluster_energy[i]- mc_part_primary_energy);}
    return out;""").Histo1D(("hh_diff_e3"+s.sn,"#Delta Ecal_e - MC Primary_e;#Delta E [GeV]",1000,-2.,1.),"d")
    s.hh_diff_xu = s.df.Define("d","""RVec<double> out;
    for(size_t i=0; i< min(ecal_cluster_uncor_x.size(),mc_score_cluster_x.size()); ++i){
        out.push_back(ecal_cluster_uncor_x[i]-mc_score_cluster_x[i]);}
    return out;""").Histo1D(("hh_diff_x"+s.sn,"Ecal_uncor_x - Score_x;#Delta x [mm]",1000,-10.,10.),"d")
    s.hh_diff_yu = s.df.Define("d","""RVec<double> out;
    for(size_t i=0; i< min(ecal_cluster_uncor_y.size(),mc_score_cluster_y.size()); ++i){
        out.push_back(ecal_cluster_uncor_y[i]-mc_score_cluster_y[i]);}
    return out;""").Histo1D(("hh_diff_y"+s.sn,"Ecal_uncor_y - Score_y;#Delta y [mm]",1000,-10.,10.),"d")
    s.hh_diff_eu = s.df.Define("d","""RVec<double> out;
    for(size_t i=0; i< min(ecal_cluster_uncor_energy.size(),mc_score_cluster_e.size()); ++i){
        out.push_back(ecal_cluster_uncor_energy[i]-mc_score_cluster_e[i]);}
    return out;""").Histo1D(("hh_diff_e"+s.sn,"Ecal_uncor_e - Score_e;#Delta E [GeV]",1000,-2.,1.),"d")
    s.hh_diff_e3u = s.df.Define("d","""RVec<double> out;
    for(size_t i=0; i< ecal_cluster_uncor_energy.size(); ++i){
        out.push_back(ecal_cluster_uncor_energy[i]- mc_part_primary_energy);}
    return out;""").Histo1D(("hh_diff_e3"+s.sn,"Ecal_uncor_e - MC Primary_e;#Delta E [GeV]",1000,-2.,1.),"d")

In [14]:
%%time
hist_colors = [R.kBlue, R.kGreen, R.kCyan, R.kBlue+1, R.kGreen+1, R.kCyan+1, R.kBlue+2, R.kGreen+2, R.kCyan+2]
hist_colors2 = [R.kOrange, R.kRed, R.kOrange+1, R.kRed+1, R.kOrange+2, R.kRed+2, R.kOrange+3, R.kRed+3]
df_store[0].hh_ecal_seed_xy.SetMinimum(0.)
df_store[0].hh_ecal_seed_xy.SetMaximum(10000)

for s  in df_store:
    s.cc0 = R.TCanvas("cc0"+s.sn,"CC0",1200,600)
    s.cc0.Divide(2,0)
    p1 = s.cc0.cd(1)
    s.leg0_p1 = R.TLegend(0.70, 0.75 ,0.99, 0.9)
    p1.SetLogy()
    s.hhn.SetStats(0)
    s.hhn2.SetStats(0)
    s.hhn2.SetTitle("Numbers")
    s.hhn2.Draw()

    s.hhn_clus.SetLineWidth(3)
    s.hhn_clus.SetLineColor(R.kCyan+1)
    s.hhn_clus.Draw("same")
    s.leg0_p1.AddEntry(s.hhn_clus.GetPtr(),"N score clusters.")

    s.hhn_ecal.SetLineWidth(3)
    s.hhn_ecal.SetLineColor(R.kRed+2)
    s.hhn_ecal.Draw("same")
    s.leg0_p1.AddEntry(s.hhn_ecal.GetPtr(),"N ecal clusters.")

    s.hhn.SetLineWidth(2)
    s.hhn.SetLineColor(R.kRed)
    s.hhn.Draw("same")
    s.leg0_p1.AddEntry(s.hhn.GetPtr(),"N Primary particles")

    s.hhn_clus_hits.SetLineColor(R.kCyan+3)
    s.hhn_clus_hits.Draw("same")
    s.leg0_p1.AddEntry(s.hhn_clus_hits.GetPtr(),"N score cluster hits.")

    s.hhn2.SetLineColor(R.kOrange)
    s.hhn2.SetStats(0)
    s.hhn2.Draw("same")
    s.leg0_p1.AddEntry(s.hhn2.GetPtr(),"N Secondary particles")
    s.hhn_hits.SetLineColor(R.kBlue)
    s.hhn_hits.Draw("same")
    s.leg0_p1.AddEntry(s.hhn_hits.GetPtr(),"N hits from primary part.")
    s.hhn2_hits.SetLineColor(R.kGreen)
    s.hhn2_hits.Draw("same")
    s.leg0_p1.AddEntry(s.hhn2_hits.GetPtr(),"N hits from secondary part.")
    s.leg0_p1.Draw()
    p2 = s.cc0.cd(2)
    p2.SetLogy()
    s.leg0_p2 = R.TLegend(0.40, 0.60 ,0.53, 0.7)
    max_y=0
    s.hhe.SetTitle(f"Energies")
    s.hhe.SetStats(0)
    s.hhe.SetLineColor(R.kGreen+1)
    s.hhe2.Draw()
    s.hhe.Draw("same")
    s.leg0_p2.AddEntry(s.hhe.GetPtr(), f"Primary E")
    s.hhe2.SetStats(0)
    s.hhe2.SetLineColor(R.kOrange)
    s.hhe2.Draw("same")
    s.leg0_p2.AddEntry(s.hhe2.GetPtr(), f"Secondaries E")

    s.cc0.cd()
    s.leg0_p2.Draw()
    s.cc0.cd()
    s.cc0_text = R.TPaveText(0.43,0.93,0.57,0.99)
    s.cc0_text.AddText(f"{s.n}")
    s.cc0_text.Draw()

    s.cc0.Draw()
    s.cc0.SaveAs("Number_and_Energies_"+s.sn+".pdf")

    s.cc0b = R.TCanvas("cc0b"+s.sn,"CC0b",1200,600)
    s.hh_ecal_seed_xy.SetStats(0)
    s.Ones = fancy_plot(s.hh_ecal_seed_xy, s.Ones, 0x0)
    s.cc0b.Draw()

CPU times: user 78.9 ms, sys: 3.82 ms, total: 82.7 ms
Wall time: 82.1 ms


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0
Info in <TCanvas::Print>: pdf file Number_and_Energies_.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0b
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0_fc
Info in <TCanvas::Print>: pdf file Number_and_Energies__fc.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0b_fc
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0_1c
Info in <TCanvas::Print>: pdf file Number_and_Energies__1c.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0b_1c
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0_fc_1c
Info in <TCanvas::Print>: pdf file Number_and_Energies__fc_1c.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc0b_fc_1c


In [15]:
cctmp = []
for i in range(len(hh_tmp_xy)):
    cctmp.append(R.TCanvas(f"cctmp{i}",f"CC {i}",1200,600))
    Ones = fancy_plot(hh_tmp_xy[i], Ones, 0x0)
    cctmp[-1].Draw()

In [16]:
%time
hist_colors = [R.kBlue, R.kGreen, R.kOrange, R.kRed, R.kCyan]
cc1 = R.TCanvas("cc1","CC1",1200,800)
cc1.Divide(2,2)
cc1.cd(1)
leg1 = R.TLegend(0.7,0.75, 0.99, 0.9)
for i in range(len(df_store)):
    s = df_store[i]
    p1 = cc1.cd(1)
    s.hh_diff_x.SetStats(0)
    s.hh_diff_x.SetLineColor(hist_colors[i])
    s.hh_diff_x.Draw("same")
    leg1.AddEntry(s.hh_diff_x.GetPtr(), f"{s.n}")
    p2 = cc1.cd(2)
    s.hh_diff_y.SetStats(0)
    s.hh_diff_y.SetLineColor(hist_colors[i])
    s.hh_diff_y.Draw("same")
    p3 = cc1.cd(3)
    p3.SetLogy()
    s.hh_diff_e.SetStats(0)
    s.hh_diff_e.SetLineColor(hist_colors[i])
    s.hh_diff_e.Draw("same")

    p4 = cc1.cd(4)
    p4.SetLogy()
    s.hh_diff_e3.SetStats(0)
    s.hh_diff_e3.SetLineColor(hist_colors[i])
    s.hh_diff_e3.Draw("same")

cc1.cd(1)
leg1.Draw()
cc1.Draw()
cc1.SaveAs("Ecal_cluster_diff_e_x_y_plots.pdf")

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 16.2 µs


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc1
Info in <TCanvas::Print>: pdf file Ecal_cluster_diff_e_x_y_plots.pdf has been created


In [17]:
%time
cc2 = R.TCanvas("cc1","CC1",1200,800)
cc2.Divide(2,2)
cc2.cd(1)
leg2 = R.TLegend(0.7,0.75, 0.99, 0.9)
for i in range(len(df_store)):
    s = df_store[i]
    p1 = cc2.cd(1)
    s.hh_diff_xu.SetStats(0)
    s.hh_diff_xu.SetLineColor(hist_colors[i])
    s.hh_diff_xu.Draw("same")
    leg2.AddEntry(s.hh_diff_xu.GetPtr(), f"{s.n}")
    p2 = cc2.cd(2)
    s.hh_diff_yu.SetStats(0)
    s.hh_diff_yu.SetLineColor(hist_colors[i])
    s.hh_diff_yu.Draw("same")
    p3 = cc2.cd(3)
    p3.SetLogy()
    s.hh_diff_eu.SetStats(0)
    s.hh_diff_eu.SetLineColor(hist_colors[i])
    s.hh_diff_eu.Draw("same")

    p4 = cc2.cd(4)
    p4.SetLogy()
    s.hh_diff_e3u.SetStats(0)
    s.hh_diff_e3u.SetLineColor(hist_colors[i])
    s.hh_diff_e3u.Draw("same")

cc2.cd(1)
leg2.Draw()
cc2.Draw()
cc2.SaveAs("Ecal_uncor_cluster_diff_e_x_y_plots.pdf")

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc1
Info in <TCanvas::Print>: pdf file Ecal_uncor_cluster_diff_e_x_y_plots.pdf has been created


In [ ]:
n_store = 3
print(f"Processing with {df_store[n_store].n}")
df_loc = df_store[n_store].df
#
# Simplify the ECAL data to a flat data set of only the 1st cluster in events with only one cluster. I.e. the resulting variables are doubles, not vector<double>
#
df_loc = df_loc.Filter("ecal_cluster_uncor_energy.size()==1 && mc_score_cluster_e.size()==1").Define("energy","ecal_cluster_uncor_energy[0]").Define("energy_cor","ecal_cluster_energy[0]")\
    .Define("x","ecal_cluster_uncor_x[0]").Define("y","ecal_cluster_uncor_y[0]") \
    .Define("x_cor","ecal_cluster_x[0]").Define("y_cor","ecal_cluster_y[0]")\
    .Define("nhits","ecal_cluster_uncor_nhits[0]").Define("seed_e","ecal_cluster_uncor_seed_energy[0]").Define("seed_ix","ecal_cluster_uncor_seed_ix[0]").Define("seed_iy","ecal_cluster_uncor_seed_iy[0]").Define("true_e","mc_part_primary_energy").Define("score_e","mc_score_cluster_e[0]").Define("score_x","mc_score_cluster_x[0]").Define("score_y","mc_score_cluster_y[0]")
cols = df_loc.AsNumpy(['energy','energy_cor','x','y','x_cor','y_cor','nhits','seed_e','seed_ix', 'seed_iy','true_e','score_e','score_x','score_y'])
print(f"Rows: {len(cols['energy'])}  Columns: {len(cols)}")
panda_df = pd.DataFrame(cols)
panda_df.to_feather("electrons_fiducial_cut_n1.feather")   # Store this Panda dataframe for easy access later.

In [13]:
panda_df.head()

,energy,energy_cor,x,y,x_cor,y_cor,nhits,seed_e,seed_ix,seed_iy,true_e,score_e,score_x,score_y
0,4.097408,4.792492,192.517929,60.816544,195.939133,60.786671,10,1.664707,12,3,4.571188,4.541945,195.154980,61.616361
1,0.996674,1.114358,-58.590794,-67.258194,-57.807549,-66.506348,4,0.829298,-8,-4,1.164342,1.152820,-58.886239,-70.651596
2,1.597344,1.834182,-143.893387,-83.234772,-147.252045,-82.997459,4,1.355381,-14,-5,1.991507,1.976017,-146.327819,-83.806891
3,3.508461,4.044655,-109.871819,-61.831448,-114.683014,-60.867279,10,1.600593,-12,-3,3.970559,3.945562,-113.936540,-61.573306
4,1.224653,1.351319,-183.952209,-79.158340,-186.979523,-78.810692,7,0.714930,-17,-5,1.483541,1.446513,-187.690922,-80.034860


In [14]:
n_store = 2
print(f"Processing with {df_store[n_store].n}")
df_loc = df_store[n_store].df
#
# Simplify the ECAL data to a flat data set of only the 1st cluster in events with only one cluster. I.e. the resulting variables are doubles, not vector<double>
#
df_loc = df_loc.Filter("ecal_cluster_uncor_energy.size()==1 && mc_score_cluster_e.size()==1").Define("energy","ecal_cluster_uncor_energy[0]").Define("energy_cor","ecal_cluster_energy[0]") \
    .Define("x","ecal_cluster_uncor_x[0]").Define("y","ecal_cluster_uncor_y[0]") \
    .Define("x_cor","ecal_cluster_x[0]").Define("y_cor","ecal_cluster_y[0]") \
    .Define("nhits","ecal_cluster_uncor_nhits[0]").Define("seed_e","ecal_cluster_uncor_seed_energy[0]").Define("seed_ix","ecal_cluster_uncor_seed_ix[0]").Define("seed_iy","ecal_cluster_uncor_seed_iy[0]").Define("true_e","mc_part_primary_energy").Define("score_e","mc_score_cluster_e[0]").Define("score_x","mc_score_cluster_x[0]").Define("score_y","mc_score_cluster_y[0]")
cols = df_loc.AsNumpy(['energy','energy_cor','x','y','x_cor','y_cor','nhits','seed_e','seed_ix', 'seed_iy','true_e','score_e','score_x','score_y'])
print(f"Rows: {len(cols['energy'])}  Columns: {len(cols)}")
panda_df = pd.DataFrame(cols)
panda_df.to_feather("electrons_n1.feather")   # Store this Panda dataframe for easy access later.
panda_df.head()

Processing with One Cluster
Rows: 3087742  Columns: 14


,energy,energy_cor,x,y,x_cor,y_cor,nhits,seed_e,seed_ix,seed_iy,true_e,score_e,score_x,score_y
0,3.553177,4.201913,-56.133678,46.464760,-59.507092,45.989395,9,2.440535,-8,2,4.090707,4.076225,-59.422582,45.784355
1,3.126054,3.552623,22.327135,-53.476910,21.488707,-52.252647,7,2.479926,-2,-3,4.141127,4.131716,23.101254,-53.667550
2,1.967187,2.165938,217.065979,-56.409370,220.501770,-55.310101,7,1.169375,13,-3,2.238227,2.222235,223.817797,-57.438733
3,2.156241,2.513875,230.833710,-83.237495,234.771042,-83.173302,4,1.710208,14,-5,2.675252,2.638022,234.629273,-87.625686
4,3.577303,4.107439,142.252625,41.217220,144.483765,40.578056,8,2.655440,8,2,4.149363,4.132494,143.307096,41.048418
